# Environment


In [1]:
# import libraries
import os
import pandas as pd  # for storing text and embeddings data
from openai import OpenAI  # for calling the OpenAI API
from openai.types.chat import ChatCompletionMessageParam
from tenacity import (
    retry,
    wait_random_exponential,
    stop_after_attempt,
)  # for retrying API calls
import tiktoken  # for counting tokens
from utils.embeddings_utils import cosine_similarity
from utils.data_utils import remove_newlines

# models
EMBEDDING_MODEL = "text-embedding-3-large"
GPT_MODEL = "ft:gpt-3.5-turbo-0125:personal:transfer-0613:9ZWWwDJS"

In [2]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# Intent Parsing


In [3]:
system_prompt = """
    You will be presented with a user intent, and your task is to extract the useful information and output it in JSON format as shown below:

    ```json
    {
    "action": "transfer",
    "chain": "Ethereum",
    "amount": "12",
    "token": "ETH",
    "receiver": "trump.eth"
    }
    ```

    If the intent is unknown, just say “I don't know”.
"""


def parse(intent: str):
    """
    Parse the user intent and return the extracted information in JSON format.
    """
    messages: list[ChatCompletionMessageParam] = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": intent},
    ]
    response = client.chat.completions.create(
        model=GPT_MODEL, messages=messages, temperature=0.7, max_tokens=500
    )
    return response.choices[0].message.content

In [4]:
parse("I want to transfer 12 ETH to trump.eth on Blast.")

'[{"action": "transfer", "chain": "Blast", "amount": "12", "token": "ETH", "receiver": "trump.eth"}]'

In [5]:
parse("Send 12 DOGE to 0x436f795B64E23E6cE7792af4923A68AFD3967952 on Ethereum.")

'[{"action": "transfer", "chain": "Ethereum", "amount": "12", "token": "DOGE", "receiver": "0x436f795B64E23E6cE7792af4923A68AFD3967952"}]'

In [6]:
parse("Send 12 PEPE and 10 USDC to apple.eth on Optimism.")

'[{"action": "transfer", "chain": "Optimism", "amount": "12", "token": "PEPE", "receiver": "apple.eth"}, {"action": "transfer", "chain": "Optimism", "amount": "10", "token": "USDC", "receiver": "apple.eth"}]'

In [7]:
parse("Send 12 PEPE to apple.eth and 10 USDT to banana.eth on Gnosis.")

'[{"action": "transfer", "chain": "Gnosis", "amount": "12", "token": "PEPE", "receiver": "apple.eth"}, {"action": "transfer", "chain": "Gnosis", "amount": "10", "token": "USDT", "receiver": "banana.eth"}]'

In [8]:
parse("Split 10 USDC and send equally to biden.eth and trump.eth on Ethereum.")

'[{"action": "transfer", "chain": "Ethereum", "amount": "5", "token": "USDC", "receiver": "biden.eth"}, {"action": "transfer", "chain": "Ethereum", "amount": "5", "token": "USDC", "receiver": "trump.eth"}]'

In [9]:
parse(
    "Send 10 USDC to vitalik.eth on Ethereum, 10 USDC to armstrong.eth on Base and 4 USDC to cz-binance.eth on BNB Chain."
)

'[{"action": "transfer", "chain": "Ethereum", "amount": "10", "token": "USDC", "receiver": "vitalik.eth"}, {"action": "transfer", "chain": "Base", "amount": "10", "token": "USDC", "receiver": "armstrong.eth"}, {"action": "transfer", "chain": "BNB", "amount": "4", "token": "USDC", "receiver": "cz-binance.eth"}]'

# Offline Outputs Generation


In [14]:
response = parse("I want to transfer 12 ETH to trump.eth on Blast.")
print(response)

[{"action": "transfer", "chain": "Blast", "amount": "12", "token": "ETH", "receiver": "trump.eth"}]


In [15]:
import json

steps = json.loads(response)
print(steps)

[{'action': 'transfer', 'chain': 'Blast', 'amount': '12', 'token': 'ETH', 'receiver': 'trump.eth'}]


In [16]:
from utils.data_utils import get_supported_actions

supported_actions = get_supported_actions()
print(supported_actions)

{'transfer': 'You are about to send {amount} {token} to {receiver} on {chain}.', 'swap': 'You are about to swap {token_in_amount} {token_in_symbol} for {token_out_amount} {token_out_symbol} on {network}.'}


In [18]:
for step in steps:
    action = step["action"]
    if action in supported_actions:
        chain = step["chain"]
        amount = step["amount"]
        token = step["token"]
        receiver = step["receiver"]
        print(
            supported_actions[action].format(
                chain=chain, amount=amount, token=token, receiver=receiver
            )
        )
    else:
        print(f"Unknown action: {action}")

You are about to send 12 ETH to trump.eth on Blast.
